# <h1> Project 1

Gradient - based optimization and differentiable programming

Pole - Cart Balancing Controller Design with Neural Network and Gradient Descent.

In [270]:
# overhead

import logging
import numpy as np
import torch as t
import torch.nn as nn
from torch.nn import utils
from torch.autograd import Variable
from torch import optim
import matplotlib.pyplot as plt

logger = logging.getLogger(__name__)

In [271]:
M = 0.5
m = 0.2
b = 0.1
I = 0.006
g = 9.8
l = 0.3
p = I*(M+m)+M*m*(l**2)

In [272]:
A = t.tensor([[0,1,0,0],[0,-(I+m*(l**2))*b/p,((m**2)*g*(l**2))/p,0],
              [0,0,0,1],[0,-(m*l*b)/p,m*g*l*(M+m)/p,0]]).float()
B = t.tensor([[0],[(I+m*(l**2))/p],
              [0],[m*l/p]]).float()
C = t.tensor([[1,0,0,0],
              [0,0,1,0]]).float()
D = t.tensor([[0],[0]]).float()

In [281]:
A = t.tensor([[1,0.009991,0.0001336,4.453e-07],
             [0,0.9982,0.02672,0.0001336],
             [0,-2.272e-05,1.002,0.01001],
             [0,-0.004544,0.3119,1.002]]).float()
B = t.tensor([[9.086e-05],
              [0.01817],
              [0.0002272],
              [0.04544]]).float()

In [273]:
class Dynamics(nn.Module):
    def __init__(self):
        super(Dynamics, self).__init__()
    @staticmethod
    def forward(state,K):
        kx = t.matmul(K,state)
        return state

In [274]:
#d = Dynamics()
#action = c.forward(s.state)
#xg = d.forward(s.state,action)

In [275]:
class Controller(nn.Module):

    def __init__(self, dim_input, dim_hidden, dim_output):
        """
        dim_input: # of system states
        dim_output: # of actions
        dim_hidden: up to you
        """
        super(Controller, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(dim_input,dim_hidden),
            nn.Tanh(),
            nn.Linear(dim_hidden,dim_output),
            nn.Sigmoid()
        )

    def forward(self, state):
        action = self.network(state)
        return action

In [276]:
class Simulation(nn.Module):

    def __init__(self, controller, dynamics, T):
        super(Simulation, self).__init__()
        self.state = self.initialize_state()
        self.controller = controller
        self.dynamics = dynamics
        self.T = T
        self.action_trajectory = []
        self.state_trajectory = []

    def forward(self, state):
        self.action_trajectory = []
        self.state_trajectory = []
        for _ in range(T):
            action = self.controller.forward(state)
            state = self.dynamics.forward(state,action)
            self.action_trajectory.append(action)
            self.state_trajectory.append(state)
            #self.state = state
        return self.error(state)

    @staticmethod
    def initialize_state():
        state = [0.2,0.,0.,0.]
        #action = [0,0,0,0]# TODO: need batch of initial states
        return t.tensor(state,requires_grad=True).float()
    def error(self,state):
        return (state[0][0]-0.2)**2 + state[0][2]**2

In [277]:
class Optimize:
    def __init__(self, simulation):
        self.simulation = simulation
        self.parameters = simulation.controller.parameters()
        self.optimizer = optim.SGD(self.parameters, lr=0.1,momentum=0.02)


    def step(self):
        print(self.simulation.controller.forward(self.simulation.state))
        #def closure():
        loss = self.simulation(self.simulation.state)
        self.optimizer.zero_grad()
        #t.nn.utils.clip_grad_norm_(self.parameters,max_norm=2)
        loss.backward()
            #return loss
        self.optimizer.step()
        self.optimizer.zero_grad()
        return loss

        #return closure()

    def train(self, epochs):
        for epoch in range(epochs):
            loss = self.step()
            print('[%d] loss: %.3f' % (epoch + 1, loss))
            #self.visualize()

    def visualize(self):
        data = np.array([self.simulation.state_trajectory[i].detach().numpy() for i in range(self.simulation.T)])
        x = data[:, 0]
        y = data[:, 1]
        plt.plot(x, y)
        plt.show()

In [278]:
#frame = 0.2
dim_input = 4
dim_output = 1
dim_hidden = 10
T = 200
d = Dynamics()  # define dynamics
c = Controller(dim_input, dim_hidden, dim_output)  # define controller
s = Simulation(c, d, T)  # define simulation
o = Optimize(s)  # define optimizer
o.train(1)  #






tensor([0.4989], grad_fn=<SigmoidBackward>)


RuntimeError: shape '[4, 1]' is invalid for input of size 16